In [1]:
from jenga.tasks.reviews import VideogameReviewsTask
from jenga.evaluation.schema_stresstest import SchemaStresstest

import tensorflow_data_validation as tfdv

import numpy as np
import pandas as pd

### Instantiate the video game reviews task with a randomly chosen seed

In [2]:
seed = np.random.randint(2**32 - 1)

task = VideogameReviewsTask(seed=seed)

### Create a tfdv schema by first auto-infering it from training data and then adjusting it

In [3]:
train_data_stats = tfdv.generate_statistics_from_dataframe(task.train_data)
schema = tfdv.infer_schema(statistics=train_data_stats)
review_date_feature = tfdv.get_feature(schema, 'review_date')
review_date_feature.distribution_constraints.min_domain_mass = 0.0

In [4]:
schema

feature {
  name: "marketplace"
  type: BYTES
  domain: "marketplace"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "customer_id"
  type: BYTES
  int_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "review_id"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_id"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_parent"
  type: BYTES
  int_domain {
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_title"
  type: BYTES
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "product_category"
  type: BYT

### Train the baseline model for the task

In [5]:
model = task.fit_baseline_model()

### Run a stress test for the schema and the model with 250 randomly chosen corruptions, and mark performance drops of 3% as failures

In [6]:
stress_test = SchemaStresstest()
results = stress_test.run(task, model, schema, num_corruptions=250, performance_threshold=.03)

SwappedValues: {'column': 'vine', 'fraction': 0.83, 'sampling': 'CAR', 'swap_with': 'verified_purchase'}
BrokenCharacters: {'column': 'title_and_review_text', 'fraction': 0.63}
SwappedValues: {'column': 'review_id', 'fraction': 0.26, 'sampling': 'CAR', 'swap_with': 'marketplace'}
SwappedValues: {'column': 'review_body', 'fraction': 0.91, 'sampling': 'CAR', 'swap_with': 'product_title'}
SwappedValues: {'column': 'marketplace', 'fraction': 0.32, 'sampling': 'CAR', 'swap_with': 'product_id'}
MissingValues: {'column': 'review_headline', 'fraction': 0.96, 'sampling': 'MAR', 'na_value': ''}
MissingValues: {'column': 'product_parent', 'fraction': 0.65, 'sampling': 'MAR', 'na_value': ''}
BrokenCharacters: {'column': 'customer_id', 'fraction': 0.58}
BrokenCharacters: {'column': 'product_parent', 'fraction': 0.54}
SwappedValues: {'column': 'review_headline', 'fraction': 0.85, 'sampling': 'CAR', 'swap_with': 'review_headline'}
BrokenCharacters: {'column': 'title_and_review_text', 'fraction': 0.27

### Look at the dataframe containing the results

In [7]:
results

,corruption,status,anomalies,baseline_score,corrupted_score
0,"SwappedValues: {'column': 'vine', 'fraction': ...",FN,{},0.792489,0.734852
1,BrokenCharacters: {'column': 'title_and_review...,FN,{},0.792489,0.739199
2,"SwappedValues: {'column': 'review_id', 'fracti...",FP,"{'marketplace': description: ""Examples contain...",0.792489,0.792489
3,"SwappedValues: {'column': 'review_body', 'frac...",TN,{},0.792489,0.792489
4,"SwappedValues: {'column': 'marketplace', 'frac...",FP,"{'marketplace': description: ""Examples contain...",0.792489,0.792489
...,...,...,...,...,...
245,"BrokenCharacters: {'column': 'product_id', 'fr...",TN,{},0.792489,0.792489
246,"GaussianNoise: {'column': 'star_rating', 'frac...",TP,"{'star_rating': description: """"\nseverity: ERR...",0.792489,0.709749
247,"BrokenCharacters: {'column': 'customer_id', 'f...",TN,{},0.792489,0.792489
248,"MissingValues: {'column': 'review_id', 'fracti...",TN,{},0.792489,0.792489
